In [1]:
import tkinter as tk
from tkinter import ttk
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import threading
import random
import time

# Setup WebDriver
def setup_driver():
    try:
        service = Service(r"C:\Users\azain\OneDrive\Desktop\chromedriver.exe")  # Replace with your ChromeDriver path
        options = webdriver.ChromeOptions()
        options.add_argument("--start-maximized")
        driver = webdriver.Chrome(service=service, options=options)
        return driver
    except Exception as e:
        print(f"Error initializing WebDriver: {e}")
        return None

# Wait for the user to manually log in
def wait_for_login(driver):
    print("Please log in to PocketOption manually. The script will continue in 30 seconds...")
    time.sleep(30)  # Wait for manual login
    print("Continuing script execution after 30 seconds.")

# Extract assets from a specific category
def extract_assets(driver, category_class):
    url = "https://pocketoption.com/en/cabinet/"  # PocketOption URL
    driver.get(url)
    
    # Wait for manual login
    wait_for_login(driver)
    
    # After login, wait for the page to load fully
    time.sleep(5)

    try:
        print(f"Attempting to locate category: {category_class}")
        # Wait for the category block to be visible
        category_block = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, category_class))
        )
        
        # Find all asset items in the category
        asset_items = category_block.find_elements(By.CLASS_NAME, "alist__item")
        print(f"Found {len(asset_items)} assets in category '{category_class}'.")  # Debugging: Check number of items found

        if len(asset_items) == 0:
            print(f"No assets found in category: {category_class}")
            return []
    except Exception as e:
        print(f"Error locating category '{category_class}': {e}")
        return []

    assets = []
    try:
        for item in asset_items:
            # Extract asset name
            name_element = item.find_element(By.CLASS_NAME, "alist__label")
            asset_name = name_element.text.strip()

            # Extract payout percentage
            payout_element = item.find_element(By.CLASS_NAME, "alist__payout")
            payout_text = payout_element.text.strip()
            assets.append({"asset": asset_name, "payout": payout_text})
            print(f"Extracted Asset: {asset_name}, Payout: {payout_text}")  # Debugging
    except Exception as e:
        print(f"Error extracting asset details: {e}")

    return assets

# Simulate lightning effect based on the number of candles
def simulate_lightning_effect(assets, candle_count):
    for asset in assets:
        # Randomly decide if the signal should be activated (simulating candles)
        if random.randint(1, 10) <= candle_count:
            asset["lightning"] = True
        else:
            asset["lightning"] = False
    return assets

# GUI Setup
def create_gui(driver):
    def update_table():
        # Get the selected category and candle count
        selected_category = category_var.get()
        selected_timeframe = timeframe_var.get()  # Retrieve selected timeframe
        try:
            candle_count = int(candle_entry.get())
        except ValueError:
            candle_count = 1  # Default to 1 if input is invalid

        # Map categories to their respective class names
        category_classes = {
            "Currencies": "assets-block__body-currency",
            "Cryptocurrencies": "assets-block__body-cryptocurrency",
            "Commodities": "assets-block__body-commodity",
            "Stocks": "assets-block__body-stock",
            "Indices": "assets-block__body-index",
        }
        category_class = category_classes.get(selected_category)

        # Fetch and update assets for the selected category
        if category_class:
            assets = extract_assets(driver, category_class)
            updated_assets = simulate_lightning_effect(assets, candle_count)

            # Clear the table
            for item in table.get_children():
                table.delete(item)

            # Populate the table with updated data
            for asset in updated_assets:
                color = "yellow" if asset.get("lightning") else "white"
                table.insert("", "end", values=(asset["asset"], asset["payout"], selected_timeframe), tags=(color,))

    def auto_update():
        update_table()
        root.after(30000, auto_update)  # Refresh every 30 seconds

    # Tkinter GUI setup
    root = tk.Tk()
    root.title("PocketOption Asset Viewer")

    # Dropdown to select asset category
    category_label = tk.Label(root, text="Select Asset Category:")
    category_label.pack(pady=10)
    category_var = tk.StringVar(value="Currencies")
    category_dropdown = ttk.Combobox(root, textvariable=category_var, state="readonly")
    category_dropdown['values'] = ["Currencies", "Cryptocurrencies", "Commodities", "Stocks", "Indices"]
    category_dropdown.pack(pady=10)

    # Dropdown for timeframes
    timeframe_label = tk.Label(root, text="Select Timeframe:")
    timeframe_label.pack(pady=10)
    timeframe_var = tk.StringVar(value="M1")
    timeframe_dropdown = ttk.Combobox(root, textvariable=timeframe_var, state="readonly")
    timeframe_dropdown['values'] = ["S5", "S10", "S15", "S30", "M1", "M2", "M3", "M5", "M10", "M15", "M30", "H1", "H4", "D1"]
    timeframe_dropdown.pack(pady=10)

    # Create a table-like view
    table_frame = ttk.Frame(root)
    table_frame.pack(fill="both", expand=True)

    table = ttk.Treeview(table_frame, columns=("Asset", "Payout", "Timeframe"), show="headings")
    table.heading("Asset", text="Asset")
    table.heading("Payout", text="Payout %")
    table.heading("Timeframe", text="Timeframe")
    table.column("Asset", width=300)
    table.column("Payout", width=100)
    table.column("Timeframe", width=100)

    # Adding coloring based on the "tags" we defined earlier
    table.tag_configure("yellow", background="yellow")
    table.tag_configure("white", background="white")
    table.pack(fill="both", expand=True)

    # Add an entry field to input the number of candles
    candle_label = tk.Label(root, text="Enter the number of candles:")
    candle_label.pack(pady=10)
    candle_entry = tk.Entry(root)
    candle_entry.pack(pady=10)
    candle_entry.insert(0, "3")  # Default to 3 candles

    # Add a refresh button
    refresh_button = tk.Button(root, text="Refresh", command=update_table)
    refresh_button.pack(pady=10)

    # Start auto-refresh
    auto_update()

    root.mainloop()

# Main Function
def main():
    driver = setup_driver()
    if driver is None:
        print("Failed to initialize WebDriver. Exiting...")
        return
    try:
        print("Launching GUI...")
        create_gui(driver)
    finally:
        driver.quit()

# Run in a separate thread
def start_bot():
    bot_thread = threading.Thread(target=main)
    bot_thread.start()

if __name__ == "__main__":
    start_bot()

Launching GUI...
Please log in to PocketOption manually. The script will continue in 30 seconds...
